# Preprocessing Data

In [8]:
import os.path
import os
import pandas as pd
import numpy as np
import censusgeocode as cg
path = os.path.abspath(os.getcwd())
data_path = os.path.abspath(os.path.join(path,'Data'))
odx_path = os.path.join(data_path,'odx')

In [9]:
stops = pd.read_csv(os.path.join(data_path,'stops.txt'))
# generating data
stop_ids = stops['stop_id'].to_numpy()
gen = 1000* np.random.normal(size=stop_ids.shape)
gen = np.int32(10000 + np.floor(gen))
stop_ids = pd.DataFrame(data = [stop_ids,gen]).T
stop_ids.columns = ['stop_id','rides']
stops = stops.merge(stop_ids)
#stops.insert(len(stops.columns),'Block',stops.apply(lambda x: geocode(x.stop_lat,x.stop_lon),axis=1))
#stops.insert(len(stops.columns),'Block',2013)
stops.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,rides
0,100148,100148,MARTIN L KING JR DR NW @ CHICAMAUGA AVE NW,33.753823,-84.431838,9043
1,210829,99971061,DEERFIELD PKWY @ TWO VERIZON PL,34.092878,-84.271101,10204
2,100150,100150,MARTIN L KING JR DR NW @ CHAPPELL RD,33.753841,-84.433273,9028
3,100152,100152,FAIR ST SW @ 1ST ST SW,33.748905,-84.425545,8900
4,100154,100154,FAIR ST@FIRST ST,33.749077,-84.425298,9386


In [16]:
(stops.stop_name == 'Five Points').sum()

0

In [18]:
routes = pd.read_csv(os.path.join(data_path,'routes.txt'))
routes.head()

,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,7634,1,Centennial Oly. Park/Coronet Way,NaN,3,NaN,819FF7,NaN
1,7635,2,Ponce de Leon Ave/Moreland Ave,NaN,3,NaN,819FF7,NaN
2,7636,3,Martin Luther King Jr. Dr/Auburn Av,NaN,3,NaN,819FF7,NaN
3,7637,4,Thomasville/Moreland Avenue,NaN,3,NaN,819FF7,NaN
4,7638,5,Piedmont Road/Sandy Springs,NaN,3,NaN,819FF7,NaN


In [22]:
stop_times = pd.read_csv(os.path.join(data_path,'stop_times.txt'))
stop_times.groupby('trip_id')

In [23]:
trips = pd.read_csv(os.path.join(data_path,'trips.txt'))
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,7634,3,5872598,Route 1 Marietta Blvd,0,321626,103952
1,7634,3,5908997,Route 1 Marietta Blvd,0,321626,103952
2,7634,3,5875737,Route 1 North Ave Station,1,322296,103898
3,7634,3,5854560,Route 1 Marietta Blvd,0,322296,103952
4,7634,3,5880985,Route 1 North Ave Station,1,321626,103898


In [37]:
stop_trips = trips.merge(stop_times, on='trip_id')
stop_trips = stop_trips.merge(routes, on='route_id')
stop_trips = stop_trips[['route_id','route_long_name','stop_id']]
stop_trips.drop_duplicates(inplace=True)
df = pd.DataFrame(pd.unique(stop_trips[stop_trips.route_long_name.str.contains("Line")].stop_id),columns=['stop_id'])
df.to_csv(os.path.join(data_path,'rail_stop_ids.csv'))


In [6]:
df = pd.read_csv(os.path.join(odx_path,'trips.csv'))
time = pd.to_datetime(df.start_time)
print(time.min())
print(time.max())

2018-01-07 03:01:00
2018-01-07 16:56:00


In [7]:
df.shape

(12849, 13)

In [38]:
census = pd.read_csv(os.path.join(data_path,'census_2010.csv'))
census = census.merge(stops,on="Block")

In [27]:
census.columns

Index(['Area Name-Legal/Statistical Area Description', 'Qualifying Name',
       'Area (Land)', 'Area (Water)', 'Summary Level', 'Geographic Component',
       'Region', 'Division', 'FIPS', 'State (FIPS)', 'County', 'Census Tract',
       'Block', 'Total Population', 'Total Population.1',
       'Total Population: Male', 'Total Population: Male: Under 5 years',
       'Total Population: Male: 5 to 9 years',
       'Total Population: Male: 10 to 14 years',
       'Total Population: Male: 15 to 17 years',
       'Total Population: Male: 18 to 24 years',
       'Total Population: Male: 25 to 34 years',
       'Total Population: Male: 35 to 44 years',
       'Total Population: Male: 45 to 54 years',
       'Total Population: Male: 55 to 64 years',
       'Total Population: Male: 65 and 74 years',
       'Total Population: Male: 75 to 84 years',
       'Total Population: Male: 85 years and over', 'Total Population: Female',
       'Total Population: Female: Under 5 years',
       'Total Pop

In [28]:
pd.unique(census.County)

array([ 63,  89, 121])